# 1. Build a convolutional neural network to classify object proposals (N+1 classes)

In [37]:
import json
from sklearn.model_selection import train_test_split
import torch
from OurDataloader import balanced_loader
from Dataset import PotholeDataset
from tqdm import tqdm
from torchvision import transforms

# Load splits.json
with open('../Potholes/splits.json', 'r') as f:
    splits = json.load(f)

# Extract train and test splits
train_files = splits['train']
test_files = splits['test']

# Further split the train set into train and validation
train_files, val_files = train_test_split(train_files, test_size=0.2, random_state=42)

In [38]:

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [39]:
# Paths
img_dir = '../Potholes/annotated-images'
annotations_dir = '../Potholes/labeled_proposals'

# Initialize datasets
train_dataset = PotholeDataset(img_dir, annotations_dir, train_files, transform=transform)
val_dataset = PotholeDataset(img_dir, annotations_dir, val_files, transform=transform)
test_dataset = PotholeDataset(img_dir, annotations_dir, test_files, transform=transform)

Loading annotations: 100%|██████████| 133/133 [00:00<00:00, 491.41it/s]


In [40]:
train_loader = balanced_loader(train_dataset, batch_size=16, weight_sample = True)
val_loader = balanced_loader(val_dataset, batch_size=16, shuffle=False, weight_sample = True)
test_loader = balanced_loader(test_dataset, batch_size=16, shuffle=False)


TypeError: balanced_loader() got an unexpected keyword argument 'weight_sample'

In [ ]:
import torch
from PotholeCNN import PotholeCNN
from TrainLoop import train
# Initialize model, optimizer, and load data
model = PotholeCNN()  # Your model class
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train the model
train(model, optimizer, epochs=10, train_loader=train_loader, test_loader=val_loader, device=device)